<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#问题实战" data-toc-modified-id="问题实战-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>问题实战</a></span><ul class="toc-item"><li><span><a href="#data-pre-process" data-toc-modified-id="data-pre-process-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>data pre-process</a></span></li><li><span><a href="#Data-exploration-analysis-of-y:-content-source" data-toc-modified-id="Data-exploration-analysis-of-y:-content-source-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data exploration analysis of y: content source</a></span></li><li><span><a href="#Word2Vec" data-toc-modified-id="Word2Vec-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Word2Vec</a></span><ul class="toc-item"><li><span><a href="#生成word2vec-model" data-toc-modified-id="生成word2vec-model-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>生成word2vec model</a></span></li><li><span><a href="#文章转换成向量" data-toc-modified-id="文章转换成向量-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>文章转换成向量</a></span></li><li><span><a href="#Try-models" data-toc-modified-id="Try-models-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Try models</a></span></li></ul></li><li><span><a href="#TFIDF" data-toc-modified-id="TFIDF-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>TFIDF</a></span><ul class="toc-item"><li><span><a href="#train-model" data-toc-modified-id="train-model-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>train model</a></span></li><li><span><a href="#抄袭文章查找" data-toc-modified-id="抄袭文章查找-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>抄袭文章查找</a></span></li></ul></li></ul></li></ul></div>

# 问题实战

问题描述： 在新闻出版业中一个常常的问题就是新闻版权抄袭，所以我们现在为了避免这个事情，需要建立一个模型，判断这个文章是不是由某个新闻出版单位出版的。 在我们这个问题里，我们需要建立一个模型，该模型接受一个作为文本的输入，然后判断该文本是不是由“新华社”发布的。 

Enviroment: 

+ Python 3.6
+ numpy 
+ scikit learning

**请在 pycharm 中运行程序，该处只作为关键信息的记录。 **

1. 问什么此问题应该用机器学习方法？ 

Ans：

2. 问什么要对文本进行向量化？ 如何进行文本向量化表示？ （请使用tfidf 或者词向量）

Ans:

hint: 如果你使用 tfidf，则需要 scikit learning 如果你需要词向量，则需要 gensim

3. 请对数据进行Preprocessing, Normalization 操作
（你需要在 Preprocssing 的时候，把文章开头的“新华社”3 个字去掉。如果不去掉，会出现什么问题？）

4. 请确定模型的 Baseline 以及确定评测指标（Evaluation）.

5. 尝试不同的模型、不同的参数，观察结果变化。 

6. 依据模型的表现，进行参数调节。 

## data pre-process

In [1]:
csv_path = 'mydata/sqlResult_1558435.csv'

In [2]:
import pandas as pd
content = pd.read_csv(csv_path, encoding='gb18030')
content = content.fillna('')

In [3]:
import jieba
def cut(string):
    return(" ".join(jieba.cut(string)))

In [4]:
cut("这是一个测试")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\CHRIST~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.075 seconds.
Prefix dict has been built succesfully.


'这是 一个 测试'

In [32]:
import re
def token(string):
    return re.findall(r'[\w|\d]+', string)

In [33]:
token("这是1个测试。\n")

['这是1个测试']

In [34]:
news_content = content['content'].tolist()

In [35]:
len(news_content)

89611

In [12]:
news_content = [token(n) for n in news_content]
news_content = [" ".join(n) for n in news_content]
news_content = [cut(n) for n in news_content]

with open("news-sentences-cut.txt", "w") as f:
    for n in news_content:
        f.write(n + '\n')

In [7]:
with open("news-sentences-cut.txt", "r") as f:
    news_content = []
    for n in f:
        news_content.append(n)

In [8]:
len(news_content)

89611

In [10]:
"新华社 abc 123".replace("新华社","")

' abc 123'

In [11]:
news_content = [i.replace("新华社","").replace("新华网","") for i in news_content]

In [12]:
news_content[0]

'此外   自 本周   6 月 12 日   起   除 小米 手机 6 等 15 款 机型 外   其余 机型 已 暂停 更新 发布   含 开发 版   体验版 内测   稳定版 暂不受 影响   以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作   有人 猜测 这 也 是 将 精力 主要 用到 MIUI   9 的 研发 之中   MIUI   8 去年 5 月 发布   距今已有 一年 有余   也 是 时候 更新换代 了   当然   关于 MIUI   9 的 确切 信息   我们 还是 等待 官方消息\n'

## Data exploration analysis of y: content source

In [13]:
len(content["source"].unique())

975

In [14]:
#content["source"]

In [15]:
content["source"].tolist().count("新华社")

78661

In [16]:
from collections import Counter

In [17]:
source_count_dic = Counter(content["source"].tolist())

In [18]:
source_count_dic["新华社"]+source_count_dic["新华网"]

78833

In [16]:
#we can also define function to count it
def dis_map(content_source = content["source"], terms = 10):
    source_list = content_source.unique().tolist()
    source_count_dic = {source: content_source.tolist().count(source) for source in source_list}
    return source_count_dic

In [45]:
source_count_dic = dis_map()

In [46]:
sorted_source_list = sorted(source_count_dic, key=lambda i: source_count_dic[i], reverse = True)

In [47]:
[(i, source_count_dic[i]) for i in sorted_source_list if source_count_dic[i] >=50]

[('新华社', 78661),
 ('微博', 2477),
 ('中国证券报?中证网', 538),
 ('中国新闻网', 525),
 ('参考消息网', 385),
 ('环球网', 308),
 ('南方日报第01版', 278),
 ('中国台湾网', 233),
 ('央广网', 197),
 ('新华网', 172),
 ('新浪体育', 132),
 ('人民网', 122),
 ('环球时报', 119),
 ('中国经济网', 96),
 ('参考军事', 95),
 ('央视网', 94),
 ('证券时报网', 84),
 ('cnBeta.COM', 82),
 ('中国证券网', 80),
 ('环球时报-环球网', 73),
 ('广州日报第01版', 64),
 ('中国网财经', 61),
 ('每日经济新闻', 58),
 ('澎湃新闻网', 54),
 ('海南日报第001版', 54),
 ('证券日报', 51),
 ('证券时报', 51),
 ('21世纪经济报道', 51)]

In [48]:
source_count_dic["新华社"]/content["source"].count()

0.87780518016761333

In [19]:
content_target = []
for i in content["source"]:
    if i == "新华社" or i =="新华网":
        content_target.append(1)
    else:
        content_target.append(0)

In [20]:
len(content_target)

89611

In [21]:
content_target[:10]

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [22]:
content["target"] = content_target

## Word2Vec

### 生成word2vec model

In [53]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

C:\Users\Christopher_luole\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [38]:
import time
startTime = time.time()
model = Word2Vec(news_content, size = 200, iter = 10, min_count = 2, workers = 4)

usedTime = time.time() - startTime
print('生成word2vec耗时%.2f秒' %usedTime)

生成word2vec耗时507.99秒


In [40]:
import pickle 

pickleFilePath = 'word2vec_model_1558435.pickle'
with open(pickleFilePath, 'wb') as file:
    pickle.dump(model, file)

In [48]:
import pickle 
# load the model
pickleFilePath = 'word2vec_model_1558435.pickle'
with open(pickleFilePath, 'rb') as file:
    model = pickle.load(file)

### 文章转换成向量

In [55]:
import numpy as np

def get_contentVector(cutWords, model):
    vector_list = [model.wv[k] for k in cutWords if k in model]
    contentVector = np.array(vector_list).mean(axis=0)
    return contentVector

In [43]:
import time

startTime = time.time()
contentVector_list = []
for i in range(len(news_content)):
    
    cutWords = news_content[i]
    if (i+1) % 2000 == 0:
        usedTime = time.time() - startTime
        print('2000文章耗时%.2f秒' %(usedTime))
        startTime = time.time()
    contentVector_list.append(get_contentVector(cutWords, model))
    
X = np.array(contentVector_list)

C:\Users\Christopher_luole\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
C:\Users\Christopher_luole\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice.
  """
C:\Users\Christopher_luole\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒
2000文章耗时0.00秒


In [44]:
txtFilePath = 'mydata/X_1558435.txt'
X.dump(txtFilePath)

In [5]:
# read the X.txt into array
import numpy as np
txtFilePath = 'mydata/X_1558435.txt'
X = np.load(txtFilePath)

In [29]:
len(X)

89611

In [38]:
X.shape

(89611,)

In [60]:
len(X[0].tolist())

200

In [61]:
sum([len(i.tolist()) for i in X if isinstance(i.tolist(), list) ]) == 200*len(X)

False

In [75]:
sum([[1,2]],[])

[1, 2]

In [83]:
np.hstack(("1",[1,2]))

array(['1', '1', '2'],
      dtype='<U11')

In [87]:
# define function to turn all array into shape (n, 200)
import numpy as np
def turn_array_shape(input_array):
    if max([len(i.tolist()) for i in input_array if isinstance(i.tolist(), list)]) >200:
        return("Error, at least one array's length over 200")
    
    output_array = np.zeros((len(input_array), 200))
    for i, value in enumerate(input_array):
        output_array[i] = np.hstack((value, [0]*200))[:200]

    #output_array = [np.hstack((value, [0]*(200-len(sum([value], []))))) for value in input_array]
    return(output_array)

In [88]:
X1 = turn_array_shape(X)

In [91]:
sum([len(i.tolist()) for i in X1 if isinstance(i.tolist(), list) ]) == 200*len(X1)

True

In [96]:
X1.shape

(89611, 200)

In [92]:
X = X1

In [131]:
X[:,0].shape

(89611,)

In [121]:
(np.matrix(X)[:,0]).shape

(89611, 1)

In [123]:
# fillna for X
# first turn X into PandasDataframe format
col_name = ["X"+str(i) for i in range(200)]

In [124]:
import pandas as pd
df_X = pd.DataFrame(columns = col_name)


In [133]:
for i, value in enumerate(col_name):
    df_X[value] = X[:,i]

In [136]:
df_X.to_csv("mydata/df_X.csv", index=False)

In [138]:
X = pd.read_csv("mydata/df_X.csv")

In [148]:
X = X.fillna(0)

In [149]:
sum(X["X1"])

-12514.085626424918

In [25]:
y = content["target"]

In [26]:
y.shape

(89611,)

In [27]:
y.reshape((1, len(y)))

C:\Users\Christopher_luole\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


array([[0, 0, 0, ..., 1, 1, 1]], dtype=int64)

In [28]:
y.count()

89611

In [29]:
sum(y)

78833

In [154]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

train_test_X, valid_X, train_test_y, valid_y = train_test_split(X, y, test_size=0.05)


In [155]:
train_X, test_X, train_y, test_y = train_test_split(train_test_X, train_test_y, test_size=0.2)


In [157]:
train_X.shape

(68104, 200)

### Try models

In [160]:
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB##GNB-- Gaussian Naive Bayes
#import lightgbm as lgb

# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.svm import LinearSVR
# from sklearn.svm import NuSVR
# from sklearn.svm import SVR


estimator_list = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    XGBClassifier(),
    #KNeighborsClassifier(),
    GaussianNB(),
]

#cv_split = ShuffleSplit(n_splits=6, train_size=0.7, test_size=0.2, random_state=2019)
df_columns = ['Name', 'Parameters', 'Train Accuracy Score', 'Test Accuracy Score', 'Valid Accuracy Score', 'Comsumed Time']
df = pd.DataFrame(columns=df_columns)

row_index = 0
for estimator in estimator_list:
    T1 = time.time()
    df.loc[row_index, 'Name'] = estimator.__class__.__name__
    df.loc[row_index, 'Parameters'] = str(estimator.get_params())
    estimator.fit(train_X, train_y)
    T2 = time.time()
    #cv_results = cross_validate(estimator, train_test_X, train_test_y, cv=cv_split)
    df.loc[row_index, 'Train Accuracy Score'] = estimator.score(train_X, train_y)
    #df.loc[row_index, 'Test Accuracy Mean'] = cv_results['test_score'].mean()
    df.loc[row_index, 'Test Accuracy Score'] = estimator.score(test_X, test_y)
    df.loc[row_index, 'Valid Accuracy Score'] = estimator.score(valid_X, valid_y)
    df.loc[row_index, 'Comsumed Time'] = round(T2-T1, 2)
    print(row_index, estimator.__class__.__name__)
    #print(cv_results['test_score'])
    row_index += 1
df = df.sort_values(by='Test Accuracy Score', ascending=False)
df

0 LogisticRegression
1 DecisionTreeClassifier
2 RandomForestClassifier
3 GradientBoostingClassifier
4 XGBClassifier
5 GaussianNB


,Name,Parameters,Train Accuracy Score,Test Accuracy Score,Valid Accuracy Score,Comsumed Time
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.971176,0.970751,0.972104,9.01
4,XGBClassifier,"{'base_score': 0.5, 'booster': 'gbtree', 'cols...",0.956464,0.950781,0.94711,185.37
3,GradientBoostingClassifier,"{'criterion': 'friedman_mse', 'init': None, 'l...",0.956141,0.9499,0.951127,162.42
2,RandomForestClassifier,"{'bootstrap': True, 'class_weight': None, 'cri...",0.998869,0.948197,0.94778,14.38
1,DecisionTreeClassifier,"{'class_weight': None, 'criterion': 'gini', 'm...",0.999956,0.927405,0.921669,43.62
5,GaussianNB,{'priors': None},0.72517,0.730765,0.728409,0.33


## TFIDF

In [17]:
def tf(word, doc):
    words = doc.split()
    return sum(1 for w in words if word == w)

In [18]:
import math
def idf(word, all_doc):
    df = sum(1 for doc in all_doc if word in doc)
    return math.log10(len(all_doc) / df)

In [19]:
tf("的", news_content[0])

2

In [20]:
idf("的", news_content) < idf("小米", news_content)

True

In [21]:
idf("的", news_content)

0.1051466115514474

We have successfully transformed the target variable and now let's turn our focus to extract features from the cleaned version of the movie plots. I have decided to go ahead with TF-IDF features. You are free to use any other feature extraction method such as Bag-of-Words, word2vec, GloVe, or ELMo. 


I recommend you check out these articles to learn more about different ways of creating features from text:



*   [An Intuitive Understanding of Word Embeddings: From Count Vectors to Word2Vec](https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/)
*   [A Step-by-Step NLP Guide to Learn ELMo for Extracting Features from Text](https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/)



In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.88, max_features=100)
#max document frequency can be tuned by various senarios, here we use 0.8 because 新华社 and not 新华社 have a ratio about 0.8:0.2


Please note that I have used the 100 most frequent words in the data as my features. You can try any other number as well for the parameter max_features. 

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

train_test_X, valid_X, train_test_y, valid_y = train_test_split(news_content, y, test_size=0.05)
train_X, test_X, train_y, test_y = train_test_split(train_test_X, train_test_y, test_size=0.2)

In [43]:
test_indices = test_y.index

In [31]:
# create TF-IDF features
import time
T1 = time.time()
xtrain_tfidf = tfidf_vectorizer.fit_transform(train_X)
xtest_tfidf = tfidf_vectorizer.transform(test_X)
xvalid_tfidf = tfidf_vectorizer.fit_transform(valid_X)
T2 = time.time()
print("耗时{}秒".format(round(T2-T1,2)))

耗时14.62秒


In [32]:
xtrain_tfidf.shape

(68104, 100)

### train model

In [33]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
def get_performance(clf, X_, y_):
    y_hat = clf.predict(X_.toarray())
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))

In [34]:
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB##GNB-- Gaussian Naive Bayes
#import lightgbm as lgb

# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.svm import LinearSVR
# from sklearn.svm import NuSVR
# from sklearn.svm import SVR


estimator_list = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    #GradientBoostingClassifier(),
    #XGBClassifier(),
    #KNeighborsClassifier(),
    #GaussianNB(),
]

#cv_split = ShuffleSplit(n_splits=6, train_size=0.7, test_size=0.2, random_state=2019)
df_columns = ['Name', 'Parameters', 'Train Accuracy Score', 'Test Accuracy Score', 'Valid Accuracy Score', 'Comsumed Time']
df = pd.DataFrame(columns=df_columns)

row_index = 0
for estimator in estimator_list:
    T1 = time.time()
    df.loc[row_index, 'Name'] = estimator.__class__.__name__
    df.loc[row_index, 'Parameters'] = str(estimator.get_params())
    estimator.fit(xtrain_tfidf, train_y)
    T2 = time.time()
    get_performance(estimator, xtest_tfidf, test_y)
    #cv_results = cross_validate(estimator, train_test_X, train_test_y, cv=cv_split)
    df.loc[row_index, 'Train Accuracy Score'] = estimator.score(xtrain_tfidf, train_y)
    #df.loc[row_index, 'Test Accuracy Mean'] = cv_results['test_score'].mean()
    df.loc[row_index, 'Test Accuracy Score'] = estimator.score(xtest_tfidf, test_y)
    df.loc[row_index, 'Valid Accuracy Score'] = estimator.score(xvalid_tfidf, valid_y)
    df.loc[row_index, 'Comsumed Time'] = round(T2-T1, 2)
    print(row_index, estimator.__class__.__name__)
    #print(cv_results['test_score'])
    row_index += 1
df = df.sort_values(by='Test Accuracy Score', ascending=False)
df

percision is: 0.9601624950858341
recall is: 0.9756974498967974
roc_auc is: 0.8363788694426687
confusion matrix: 
[[ 1399   608]
 [  365 14654]]
0 LogisticRegression
percision is: 0.973835268989454
recall is: 0.9714361808376057
roc_auc is: 0.888059894105898
confusion matrix: 
[[ 1615   392]
 [  429 14590]]
1 DecisionTreeClassifier
percision is: 0.976286679472743
recall is: 0.9813569478660363
roc_auc is: 0.9014906313819468
confusion matrix: 
[[ 1649   358]
 [  280 14739]]
2 RandomForestClassifier


,Name,Parameters,Train Accuracy Score,Test Accuracy Score,Valid Accuracy Score,Comsumed Time
2,RandomForestClassifier,"{'bootstrap': True, 'class_weight': None, 'cri...",0.993686,0.962528,0.805847,5.24
1,DecisionTreeClassifier,"{'class_weight': None, 'criterion': 'gini', 'm...",0.995008,0.95178,0.720152,4.42
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.942147,0.942852,0.829279,0.51


In [35]:
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB##GNB-- Gaussian Naive Bayes
#import lightgbm as lgb

# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.svm import LinearSVR
# from sklearn.svm import NuSVR
# from sklearn.svm import SVR


estimator_list = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    XGBClassifier(),
    #KNeighborsClassifier(),
    #GaussianNB(),
]

#cv_split = ShuffleSplit(n_splits=6, train_size=0.7, test_size=0.2, random_state=2019)
df_columns = ['Name', 'Parameters', 'Train Accuracy Score', 'Test Accuracy Score', 'Valid Accuracy Score', 'Comsumed Time']
df = pd.DataFrame(columns=df_columns)

row_index = 0
for estimator in estimator_list:
    T1 = time.time()
    df.loc[row_index, 'Name'] = estimator.__class__.__name__
    df.loc[row_index, 'Parameters'] = str(estimator.get_params())
    estimator.fit(xtrain_tfidf, train_y)
    T2 = time.time()
    #get_performance(estimator, xtest_tfidf, test_y)
    #cv_results = cross_validate(estimator, train_test_X, train_test_y, cv=cv_split)
    df.loc[row_index, 'Train Accuracy Score'] = estimator.score(xtrain_tfidf, train_y)
    #df.loc[row_index, 'Test Accuracy Mean'] = cv_results['test_score'].mean()
    df.loc[row_index, 'Test Accuracy Score'] = estimator.score(xtest_tfidf, test_y)
    df.loc[row_index, 'Valid Accuracy Score'] = estimator.score(xvalid_tfidf, valid_y)
    df.loc[row_index, 'Comsumed Time'] = round(T2-T1, 2)
    print(row_index, estimator.__class__.__name__)
    #print(cv_results['test_score'])
    row_index += 1
df = df.sort_values(by='Test Accuracy Score', ascending=False)
df

0 LogisticRegression
1 DecisionTreeClassifier
2 RandomForestClassifier
3 GradientBoostingClassifier
4 XGBClassifier


,Name,Parameters,Train Accuracy Score,Test Accuracy Score,Valid Accuracy Score,Comsumed Time
2,RandomForestClassifier,"{'bootstrap': True, 'class_weight': None, 'cri...",0.993671,0.961353,0.751172,5.49
4,XGBClassifier,"{'base_score': 0.5, 'booster': 'gbtree', 'cols...",0.954966,0.953131,0.778398,14.86
3,GradientBoostingClassifier,"{'criterion': 'friedman_mse', 'init': None, 'l...",0.953365,0.952778,0.546976,29.41
1,DecisionTreeClassifier,"{'class_weight': None, 'criterion': 'gini', 'm...",0.995008,0.951545,0.657666,4.36
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.942147,0.942852,0.829279,0.37


### 抄袭文章查找

In [36]:
# we choose the model with highest Valid Accuracy Score - Logistic Regression
clf = LogisticRegression()
clf.fit(xtrain_tfidf, train_y)
y_hat_test = clf.predict(xtest_tfidf.toarray())
#test_y

In [44]:
candidate_indices = []
for index, (y,yhat) in enumerate(zip(test_y,y_hat_test)):
    if y == 0 and yhat == 1:
        candidate_indices.append(test_indices[index])

In [54]:
candidate_indices[:5]

[5897, 7705, 6956, 685, 8031]

In [47]:
content.iloc[5897]['source']

'新浪体育'

In [48]:
IP_violator = [content.iloc[i]['source'] for i in candidate_indices]


In [49]:
from collections import Counter

In [52]:
sorted_source_list = sorted(Counter(IP_violator), key=lambda i: Counter(IP_violator)[i], reverse = True)

In [56]:
[(i, Counter(IP_violator)[i]) for i in sorted_source_list if Counter(IP_violator)[i] >=5]

[('中国新闻网', 90),
 ('参考消息网', 41),
 ('南方日报第01版', 23),
 ('央广网', 17),
 ('环球时报', 16),
 ('新浪体育', 15),
 ('环球网', 14),
 ('人民网', 13),
 ('环球时报-环球网', 9),
 ('湖北日报第1版', 9),
 ('中国台湾网', 8),
 ('中国经济网', 8),
 ('中国证券报?中证网', 8),
 ('郑州日报第03版', 7),
 ('参考军事', 7),
 ('经济参考报', 6),
 ('河南日报第03版', 5),
 ('证券时报', 5),
 ('成都商报第01版', 5),
 ('央视网', 5),
 ('中国网', 5),
 ('长江日报第1版', 5),
 ('郑州日报第08版', 5),
 ('凤凰体育', 5),
 ('郑州日报第02版', 5),
 ('每日经济新闻', 5)]